In [6]:
!pip install python-dotenv
!pip install openai

# Getting data from Mergeflow's API

In [7]:
import requests
import json
import os
from dotenv import load_dotenv # stores API keys for OpenAI and Mergeflow 

load_dotenv()  # Load variables from .env file
mergeflow_api_key = os.getenv('MERGEFLOW_API_KEY') # Assign Mergeflow API

# the API call
query = 'factory automation' # topic is factory automation
dataset = '&q=*%5bfundingcompany%7cfundingproject%7cfundingorganization%5d&sp=2572' # access dataset from mergeflow - in this case venture capital investments
rows = str(10) # first 10 docs only

# in the url below, replace "fw" by your user name. You can see your user name at mergeflow.net
# -> Account -> the string in angle brackets
url = 'https://mergeflow.net/api/v1/fw/search/getContent?q=' + query + dataset + '&rows=' + rows

# Define the headers to be sent with the request
headers = {
    'MergeflowNet-Api-Auth-Key': mergeflow_api_key
}

# Get and print the response
response = requests.post(url, headers=headers)

mergeflow_docs = []

# Check the response status code and content
if response.status_code == 200:
    mergeflow_response_json = response.json()
    
    # Pretty-print the JSON response -- useful to understand what the JSON looks like
    #pretty_json = json.dumps(mergeflow_response_json, indent=4)
    #print(pretty_json)
    
    # get title, URL, date, and content
    for document in mergeflow_response_json['Documents']:
        current_doc = {
            'title': document['Title'],
            'date': document['Date'],
            'url': document['Url'],
            'content': document['Content']
        }
        
        mergeflow_docs.append(current_doc)
        #print(content)        

    print('Found ' + str(len(mergeflow_docs)) + ' documents.')

else:
    print('Request failed with status code:', response.status_code)
    print(response.text)  # Print the response text


Found 10 documents.


In [8]:
import openai
import os
import requests
import json

openai.api_key = os.getenv('OPENAI_API_KEY')

summary_list = []

system_prompt_summary = f"""
You are a highly qualified academic scholar who has much experience in communicating your results to a broad audience. You are able to extract the most important aspects of texts, compress them and rephrase them in a comprehensible way.
"""

for doc in mergeflow_docs:
    user_prompt_summary = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {doc['content']}
    ```
    Please summarize the finding of the text in at most 500 words, and clearly cite the source at the end.
    """
    completion_summary = openai.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "system", "content": system_prompt_summary},
            {"role": "user", "content": user_prompt_summary}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

    summary_list.append(completion_summary.choices[0].message.content)
    print(completion_summary.choices[0].message.content)
    print("\n---\n")


print(summary_list)
print("\n----------------------------------------------\n----------------------------------------------\n")


Aniai, a New York City-based robotic company specializing in kitchen solutions, recently secured $12 million in Pre-Series A funding. The funding round was spearheaded by InterVest, with contributions from SV Investment, Ignite Innovation from the UK, and Capstone Partners, an existing investor. The primary goal of this financial injection is to establish Aniai's first manufacturing facility named Factory One in South Korea. This move aims to ensure stable production and enhance supply chain management for their flagship product, the Alpha Grill. The company's strategic focus aligns with its ongoing expansion efforts in North America and globally.

Founded in 2020 by CEO Gunpil Hwang, Aniai focuses on developing robotic solutions tailored for cooking hamburgers. The Alpha Grill, their leading product, employs robotics to efficiently cook hamburgers, featuring a double-sided grill capable of cooking up to eight patties simultaneously or 200 patties per hour. By automating tasks such as 

In the field of factory automation, Numbers Station, an intelligent data stack automation platform based in Menlo Park, CA, has recently secured significant funding. The company announced a successful $17.5 million funding round, comprising a $12.5 million series A round led by Madrona and supported by investors like Norwest Venture Partners, Factory, and notable angel investors including Jeff Hammerbacher, a co-founder of Cloudera. This series A funding follows a previous $5 million seed round, bringing the total capital raised by Numbers Station to $17.5 million. The platform utilizes foundation model technology developed at the Stanford AI lab to automate data work processes, particularly focusing on data transformation tasks. This automation enables data workers to efficiently generate insights by streamlining tedious data-related tasks.

Source: Numbers Station press release, MENLO PARK, CA.

---

Numbers Station, a Menlo Park, CA-based company, has secured $17.5M in funding for i

Worlds Enterprises, Inc. has secured $21.2 million in Series A1 funding to further develop its Industrial Metaverse platform, which integrates AI-based automation into the operations of large industrial companies. The funding round was led by Moneta Ventures and saw participation from existing investors like Align Capital, Green Park & Golf Ventures, and Chevron Technology Ventures. This financial boost will support the company in enhancing its AI platform, expanding partnerships, and increasing the adoption of its technology within the industrial sector.

The Industrial Metaverse platform by Worlds creates a 4D infrastructure that enables the convergence of IoT sensors, people, and processes into a digital twin, allowing organizations to analyze and optimize their operations in ways previously unattainable. CEO Dave Copps expressed excitement over the shared vision for digital transformation with investors and highlighted the company's role in facilitating the merging of digital and p

In [9]:
Problem = """Based on the provided information, the problem with the highest overall rating is:
    - Problem: Lack of seamless integration and communication between different factory automation systems and equipment.
    - User Group Impacted: Factory automation engineers and operators.
    - Specific Pain Point: Inefficient workflows, increased downtime, and suboptimal performance due to manual intervention and lack of real-time data synchronization.
    - Summary of Selection Criteria: Large target group impacted, very relevant pain point to factory automation industry and users, specific and critical issue in the industry."""

print(Problem)

Based on the provided information, the problem with the highest overall rating is:
    - Problem: Lack of seamless integration and communication between different factory automation systems and equipment.
    - User Group Impacted: Factory automation engineers and operators.
    - Specific Pain Point: Inefficient workflows, increased downtime, and suboptimal performance due to manual intervention and lack of real-time data synchronization.
    - Summary of Selection Criteria: Large target group impacted, very relevant pain point to factory automation industry and users, specific and critical issue in the industry.


# Answering questions on the results with OpenAI GPT

In [10]:
import openai

openai.api_key = os.getenv('OPENAI_API_KEY') # Assign OpenAI API

#System prompt (for all users)
system_prompt = f"""You are Senior Vice President Technology and Innovation & CTO of Factory Automation. When you evaluate a new business idea, you strictly adhere to the following guiding principles:
 
        {Problem}
        
        - Prioritize quick, low-investment idea testing.
        - Aim for large profits with smaller budgets
        - Avoid new ecosystem creation.
        - Prioritize product over solution business for scalability.
        - Consider AI as a potential product.
        - Ensure all initiatives align with business profitability.
        - Aim for non-cannibalizing, fringe markets.
        - Validate ideas with business units and customer feedback.
        - Avoid focusing solely on infrastructure to ensure profitable outcomes.
        """

summary_solutions = []

user_prompt = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {doc['content']}
    ```

    You want to generate a technology roadmap for the referred industry. Kindly follow these guidelines
    - Be realistic and specific
    - mention a specific trends or products at the beginning in your output
    - Iterate the same trend/product through years
    - Focus and elaborate more on the technology features
    - Have yearly plan for the trends/products technology advancement
    - Do consider about challenges in business when making decision
    
    
    """
    
completion = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.8,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )


print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
summary_solutions.append(completion.choices[0].message.content)

### Technology Roadmap for Factory Automation: Focusing on Industrial-Control-as-a-Service (ICaaS)

#### Year 1: Foundation and Integration

**Trend/Product:** Launch of ICaaS with a focus on cloud-based management of PLCs, Git-enabled PLC code versioning, and virtualization of PLCs on edge servers.

- **Q1-Q2:** Begin with the development and integration of cloud-based management for existing Programmable Logic Controllers (PLCs). This phase focuses on enabling remote TechOps to manage factory automation systems more efficiently.
- **Q3:** Implement Git-enabled PLC code versioning and collaboration tools. This will facilitate DevOps practices in the industrial automation sector, promoting better code management and collaboration among engineers.
- **Q4:** Start the virtualization of PLCs on edge servers. This will reduce the dependency on physical hardware and enable more flexible and scalable automation systems.

**Challenges:** Ensuring seamless integration with existing factory aut

In [12]:
# Initial rating of the solutions

initial_rating = []

system_prompt_2 = f"""You are a highly qualified critical rater of business opportunities. You are now tasked to evaluate a set of opportunities with the following problem in mind: {Problem} """


user_prompt4 = f""""

First, repeat the following points for the first solution as found in the following text: {summary_solutions}

    - "Name"
    - “Solved Problem”
    - “Solution”
    - “Impact”
    - “Technology”
    - “Sources”

Then, rate the first solution according to the following criteria. Please justify your ratings in max one sentence. Please be very critical in your ratings and only give a 3 out of 3 rating when it is very good:   

    - Functionality and Performance: Assess whether the solution addresses and solves the problem it’s supposed to solve. This includes estimating if the solution can solve the problem quickly and effectively. (1 = low performance, 2 = medium performance, 3 = high performance)
    - Scalability: Evaluate whether the solution can adapt to future growth (or shrinking) of demand and accommodate changes in requirements: How flexible is the solution? (1 = low scalability, 2 = medium scalability, 3 = high scalability)
    - Cost-effectiveness: Assess the costs associated with the solution. How large are upfront costs for implementation such as training, facilities, machines, etc.? How large are operating costs such as personnel, energy, maintenance, etc.? The solution should have a relatively high return on investment. (1 = low cost-effectiveness, 2 = medium cost-effectiveness 3 = high cost-effectiveness)
    - Innovativeness: How innovative is the solution? Do similar solutions for the problem exist already? (1 = low innovativeness, 2 = medium innovativeness 3 = high innovativeness)
    - Uniqueness: The solutions contained in the list should differ substantially and not be similar. (1 = low uniqueness, 2 = medium uniqueness 3 = high uniqueness)
    - Overall Rating: Sum of all the previous ratings, out of 15

Now, repeat these steps for all solutions.

"""


completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt_2},
{"role": "user", "content": user_prompt4}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
initial_rating.append(completion.choices[0].message.content)


### Technology Roadmap for Factory Automation: Focusing on Industrial-Control-as-a-Service (ICaaS)

- **Name:** Industrial-Control-as-a-Service (ICaaS)
- **Solved Problem:** Lack of seamless integration and communication between different factory automation systems and equipment.
- **Solution:** A roadmap for the development and growth of ICaaS, including cloud-based management of PLCs, Git-enabled PLC code versioning, virtualization of PLCs on edge servers, optimization based on feedback, expansion of virtual PLC functionality with AI for predictive maintenance, standardization across industries, and continuous innovation.
- **Impact:** Improved efficiency, reduced downtime, and enhanced performance through seamless integration and real-time data synchronization.
- **Technology:** Cloud-based management, Git-enabled versioning, virtualization, AI and machine learning.
- **Sources:** Not provided.

#### Ratings:

- **Functionality and Performance:** 3 - The detailed roadmap suggests a 

In [13]:
user_prompt_3 = f""""

Please take all the solutions and the corresponding ratings from: {initial_rating}

Take the three solutions with the highest absolute rating out of 15 (please be very diligent in this task. If there is an overall rating of 13, it should for example be outputed instead of an idea with a score of 10 out of 15). Sort the three in descending rating order.

Repeat the followinng points for each of the top three:

    - "Name"
    - “Solved Problem”
    - “Solution”
    - “Impact”
    - “Technology”
    - “Sources”
    - also include their rating from: {initial_rating} to double check they are in the top 3 highest rated


"""


completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt_2},
{"role": "user", "content": user_prompt_3}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability

Based on the provided information, there is only one solution detailed, which is the Industrial-Control-as-a-Service (ICaaS). Since there are no other solutions provided to compare against, the ICaaS solution is by default the highest-rated option given, with an overall rating of 14 out of 15. Here are the details for the ICaaS solution:

- **Name:** Industrial-Control-as-a-Service (ICaaS)
- **Solved Problem:** Lack of seamless integration and communication between different factory automation systems and equipment.
- **Solution:** A roadmap for the development and growth of ICaaS, including cloud-based management of PLCs (Programmable Logic Controllers), Git-enabled PLC code versioning, virtualization of PLCs on edge servers, optimization based on feedback, expansion of virtual PLC functionality with AI for predictive maintenance, standardization across industries, and continuous innovation.
- **Impact:** Improved efficiency, reduced downtime, and enhanced performance through seamless